# Knickpoint Analyses for Geoscience
-Nick Lewis, Joanmarie Del Vecchio, Chuck Bailey.

Steps
- Get Dem (take user bounds, res.)
- Download geology shp files
- Download WBD GDB
- User can now select what they want to do.
Export files include:
	- Folders for each watershed w name
        - Relief map
            - W/ LSHT highlighted
	    - Slope map (similar but not the same!)
            - LSHT
	    - Knickpoints mapped (maybe adjust delta ksn?  To isolate bedrock reaches.)
		- Profile w trimodal morphology represented (maybe, refer to chuck suggestion)
	- CSVs of knickpoints
		- To be included:
        - Wshed
		- elevation
		- Coords
		- Shapefile geology
		- Nearest roadcut geology that is also within some threshold of elevation from knickpoint.  Do buffer around knickpoint select closest to elevation.
		- Stream cover type (to allow narrowing to bedrock)
		- If BR, fracture data, if AP, bedding
		- If not in AP or BR, Send back a message being like sorry dawg.
		- Depth to mantle @ knickpoint.  Z score to average depth in region.
		- Slope

# Code begins here!

#### Below is a 'code cell'.  This is a block of text that Jupyter and Python use to perform tasks.

#### To run the following code cell and all code cells below, select the cell and press
# <p style="color: aqua;">Shift+Enter.</p>

In [ ]:
# This is called a comment.  Denoted by the '#' behind the text, it allows me to better explain code cells while still being a begign line of text.

# First, we import the YML file.  This saves you the work of making your own python environment and installing a bunch of packages.
#!conda env create -f env.yml

In [1]:
# From google drive, let's get the peripheral files our code will use.
#function here that imports shit

In [ ]:
# Import packages to our notebook.
# We must do this to use the functions contained inside the packages.
# Some functions have 'as' in the import call.  This is for less typing, and nothing else.
# Google any package name and 'docs' in order to find more detailed explanation of any one package.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import xdem
import os
import rasterio as rio
from rasterio.plot import show
import glob
from concurrent.futures import ThreadPoolExecutor

#### Phew!  That was a lot of initial stuff, but no worries.  We can start *really* coding now.

#### Below is a code cell containing our **Constant** Variables, which you may change as you see fit for your particular analyses.

#### Below are 
- Integers
- Lists (denoted by square brackets at the beginning and end, separated by commas [a, b, c, d])
- Strings (contained within quotes 'Like this!')

In [6]:
# Every time you change something within the code cell, run it once more to update the variable.

BOUNDS = [1.0, 2.0, 3.0, 4.0] # In decimal coordinate form.  Format: [N, E, S, W].  Select the smallest area possible in your analyses for maximum resoultion!
API_KEY = 'placeholder' # Get your API key from opentopography.org.  Students get higher resolution data!
DEM_DATASET = 'USGS30m' # Options are USGS30m, USGS10m, & USGS1m.  Be warned!  As resolution goes up, available area goes down.
WBD_SCALE = 8 # Size threshold of watersheds your DEM will be split into.  From largest to smallest, the options are 4, 6, 8, & 10.  See below for examples of scale.
PROJ_TITLE = 'placeholder' # Create a succinct name with no spaces or leading digits to represent your project file for future exports.